In [42]:
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Dropout, Flatten, Embedding, Flatten, Dense, LSTM
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D
from tensorflow.keras.preprocessing import sequence
import tensorflow.keras.utils
import numpy as np
import pandas as pd
import spacy
import spacy_transformers
import en_trf_bertbaseuncased_lg
import os

import tqdm

In [2]:
df = pd.read_csv('train.csv')

In [23]:
df.dropna(inplace=True)

In [4]:
df_test = pd.read_csv('test.csv')

In [5]:
df_test

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive
...,...,...,...
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative
3530,416863ce47,All alone in this old house again. Thanks for...,positive
3531,6332da480c,I know what you mean. My little dog is sinkin...,negative
3532,df1baec676,_sutra what is your next youtube video gonna b...,positive


In [33]:
def clean_my_text(text):
    lemmatized = []
    text = text.lower()
    tokens = model(text)
    for word in tokens:
        if not word.is_stop and word.is_alpha:
            lemmatized.append(word.lemma_)
    return lemmatized

In [32]:
model = spacy.load('en_trf_bertbaseuncased_lg')

In [ ]:
tok2vec = nlp.create_pipe("trf_tok2vec")

In [34]:
X = df['text']
y = df['sentiment']

In [35]:
y.replace({'positive':1, 'neutral':0, 'negative':-1}, inplace=True)

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [36]:
y = tensorflow.keras.utils.to_categorical(y, num_classes=3, dtype='float32')

In [37]:
clean_X = []
for text in tqdm.tqdm(X):
    results = clean_my_text(text)
    clean_X.append(results)

100%|██████████| 27480/27480 [1:05:46<00:00,  6.96it/s]


In [45]:
vocab_list = ['']
for text in clean_X:
    for word in text:        
        vocab_list.append(word)

# takes care of the duplicates
vocab_list = list(set(vocab_list))

In [65]:
word_to_num = {}
num_to_word = {}
for i, word in enumerate(vocab_list):
    num_to_word[i] = word
    word_to_num[vocab_list[i]] = i

In [66]:
word_vec_x = [[word_to_num[word] for word in text] for text in clean_X]

In [48]:
max_lenght = len(sorted(clean_X)[-1])

In [49]:
clean_X

[['responded', 'going'],
 ['sooo', 'sad', 'miss', 'san', 'diego'],
 ['boss', 'bullying'],
 ['interview', 'leave'],
 ['sons', 'releases', 'bought'],
 ['shameless', 'plugging', 'best', 'rangers', 'forum', 'earth'],
 ['feedings', 'baby', 'fun', 'smiles', 'coos'],
 ['soooo', 'high'],
 [],
 ['journey', 'wow', 'u', 'cooler', 'hehe', 'possible'],
 ['love',
  'hopeful',
  'reckon',
  'chances',
  'minimal',
  'p',
  'go',
  'to',
  'cake',
  'stuff'],
 ['like', 'song', 'love', 'story', 'taylor', 'swift'],
 ['sharpie', 'running', 'dangerously', 'low', 'ink'],
 ['want', 'music', 'tonight', 'lost', 'voice'],
 ['test', 'test', 'lg'],
 ['uh', 'oh', 'sunburned'],
 ['trying', 'plot', 'alternatives', 'speak', 'sigh'],
 ['sick', 'past', 'days', 'hair', 'looks', 'wierd', 'not', 'hat', 'look'],
 ['home', 'go', 'to', 'miss'],
 ['s'],
 ['oh', 'marly', 'sorry', 'hope', 'find', 'soon'],
 ['playing',
  'ghost',
  'online',
  'interesting',
  'new',
  'updates',
  'kirin',
  'pet',
  'metamorph',
  'job',
  'w

In [16]:
len(word_vec_x[0])

2

In [67]:
word_vec_X = sequence.pad_sequences(word_vec_x, maxlen = 22, padding='pre' )

In [51]:
len(word_vec_x)

27480

In [52]:
max_lenght

7

In [24]:
clean_x_series = pd.Series(clean_X)

In [25]:
clean_x_series = clean_x_series.apply(pd.Series)

In [26]:
clean_x_series.dropna(axis=0)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
1234,xo,hey,chick,u,alryt,u,dads,tmoro,sud,sumin,...,week,neva,dun,oot,week,lol,missed,ha,bye,hun
16149,av,ad,realy,gd,day,wiv,ciara,connolly,park,gate,...,new,brighton,fukin,funny,bt,sunburnt,luks,like,drivers,arm


In [70]:
pd.DataFrame(word_vec_X)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,19311,3180
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,11377,10074,2917,18981,12147
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15245,18474
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,423,8774
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,13675,15678,14154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27475,0,0,0,0,0,0,0,0,0,0,...,0,0,12119,4364,17178,15451,15494,5916,6850,16641
27476,0,0,0,0,0,0,0,0,0,0,...,10531,3502,22071,13292,14926,15539,17516,17634,1300,13967
27477,0,0,0,0,0,0,0,0,0,0,...,4223,19659,17559,17711,20899,10484,4434,10226,13172,10106
27478,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9225


In [71]:
pd.DataFrame(word_vec_X).to_csv("word_vec_X_trans.csv", index=None)

In [72]:
word_vec_X = pd.read_csv('word_vec_X_trans.csv')


In [83]:
word_vec_X = word_vec_X.to_numpy()

In [84]:
word_vec_X.shape

(27480, 22)

In [85]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(word_vec_X, y)

In [86]:
vocab_size = len(vocab_list) + 1 
vocab_size

22735

In [31]:
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=22))
model.add(Dropout(0.5))
model.add(BatchNormalization())
# model.add(LSTM(1024))
# model.add(Dropout(0.5))
model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(1, activation='sigmoid'))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.35))
model.add(BatchNormalization())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.35))
model.add(BatchNormalization())

model.add(Dense(3, activation='softmax'))

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 22, 64)            1455040   
_________________________________________________________________
dropout (Dropout)            (None, 22, 64)            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 22, 64)            256       
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1181696   
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2

In [33]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

In [35]:
model.fit(Xtrain, ytrain, epochs=3, batch_size=32, validation_split=0.2)

Train on 16488 samples, validate on 4122 samples
Epoch 1/3
16488/16488 [==============================] - 28s 2ms/sample - loss: 0.6935 - accuracy: 0.6562 - val_loss: 0.5241 - val_accuracy: 0.7196
Epoch 2/3
16488/16488 [==============================] - 23s 1ms/sample - loss: 0.4854 - accuracy: 0.7705 - val_loss: 0.4452 - val_accuracy: 0.7935
Epoch 3/3
16488/16488 [==============================] - 23s 1ms/sample - loss: 0.3969 - accuracy: 0.8261 - val_loss: 0.4614 - val_accuracy: 0.8017


In [36]:
from sklearn.metrics import classification_report

y_pred = model.predict(Xtrain, batch_size=64, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
ytrain_bool = np.argmax(ytrain, axis=1)
print(classification_report(ytrain_bool, y_pred_bool))

              precision    recall  f1-score   support

           0       0.79      0.81      0.80      8362
           1       0.86      0.83      0.85      6477
           2       0.81      0.81      0.81      5771

    accuracy                           0.82     20610
   macro avg       0.82      0.82      0.82     20610
weighted avg       0.82      0.82      0.82     20610



In [37]:
y_pred = model.predict(Xtest, batch_size=64, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
ytest_bool = np.argmax(ytest, axis=1)
print(classification_report(ytest_bool, y_pred_bool))

              precision    recall  f1-score   support

           0       0.65      0.70      0.68      2755
           1       0.78      0.73      0.76      2105
           2       0.70      0.67      0.68      2010

    accuracy                           0.70      6870
   macro avg       0.71      0.70      0.71      6870
weighted avg       0.70      0.70      0.70      6870



### BERT

In [76]:
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

In [77]:
model_dir = "/storage/sentiment/uncased_L-12_H-768_A-12"

bert_params = bert.params_from_pretrained_ckpt(model_dir)
l_bert = bert.BertModelLayer.from_params(bert_params, name="bert")

In [101]:
max_seq_len = 22
l_input_ids      = tensorflow.keras.layers.Input(shape=(max_seq_len,), dtype='int32')
# l_token_type_ids = tensorflow.keras.layers.Input(shape=(max_seq_len,), dtype='int32')

# using the default token_type/segment id 0
output = l_bert(l_input_ids) # output: [batch_size, max_seq_len, hidden_size]

cls_out = tensorflow.keras.layers.Lambda(lambda seq: seq[:, 0, :])(output)
cls_out = Dropout(0.5)(cls_out)
# cls_out = BatchNormalization()(cls_out)
logits = Dense(768, activation="tanh")(cls_out)
logits = Dropout(0.5)(logits)
# logits = Dense(128, activation="relu")(logits)
# logits = Dropout(0.5)(logits)
logits = Dense(units=3,activation="softmax")(logits)


model = tensorflow.keras.Model(inputs=l_input_ids, outputs=logits)
model.build(input_shape=(None, max_seq_len))


# model = tensorflowkeras.Model(inputs=l_input_ids, outputs=output)
# model.build(input_shape=(None, max_seq_len))

# provide a custom token_type/segment id as a layer input
# output = l_bert([l_input_ids, l_token_type_ids])          # [batch_size, max_seq_len, hidden_size]
# model = tensorflow.keras.Model(inputs=[l_input_ids, l_token_type_ids], outputs=output)
# model.build(input_shape=[(None, max_seq_len), (None, max_seq_len)])

In [102]:
bert_ckpt_file   = os.path.join(model_dir, "bert_model.ckpt")
bert.load_stock_weights(l_bert, bert_ckpt_file)

Done loading 196 BERT weights from: /storage/sentiment/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7fd90b3334e0> (prefix:bert_1). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


[]

In [103]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics= ['accuracy'])

In [104]:
model.fit(Xtrain, ytrain, epochs=5, batch_size=128, validation_split=0.2)

Train on 16488 samples, validate on 4122 samples
Epoch 1/5
16488/16488 [==============================] - 131s 8ms/sample - loss: 0.6859 - accuracy: 0.6290 - val_loss: 0.6071 - val_accuracy: 0.6773
Epoch 2/5
16488/16488 [==============================] - 98s 6ms/sample - loss: 0.6293 - accuracy: 0.6672 - val_loss: 0.5608 - val_accuracy: 0.7158
Epoch 3/5
 6272/16488 [==========>...................] - ETA: 55s - loss: 0.5920 - accuracy: 0.6897

KeyboardInterrupt: 

In [217]:
from transformers import BertTokenizer

In [42]:
import os